<a href="https://colab.research.google.com/github/sujithrc/NER_using_Spcacy/blob/main/NER_using_spacy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Read the text file line by line
with open('/content/sample_data/para.txt', 'r', encoding='ISO-8859-1') as file:
    lines = file.readlines()

# Create a DataFrame from the list of lines
df = pd.DataFrame(lines, columns=['Text'])

# Remove any trailing newline characters
df['Text'] = df['Text'].str.strip()

# Display the first few rows of the DataFrame
print(df.head())



             Text
0        %PDF-1.4
1           %öäüß
2         1 0 obj
3              <<
4  /Type /Catalog


In [ ]:

# Install spacy and the transformer model

!pip install spacy[transformers]

!python -m spacy download en_core_web_trf



/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.7.3/en_core_web_trf-3.7.3-py3-none-any.whl (457.4 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
print(spacy.__version__)  # Should be 3.0 or higher




3.7.6


In [ ]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
import os

# Sample training data (you can expand this)
train_data = [
    ("Apple is a tech company.", {"entities": [(0, 5, "ORG")]}),
    ("London is the capital of England.", {"entities": [(0, 6, "GPE")]}),
    ("Barack Obama was the 44th president of the United States.", {"entities": [(0, 12, "PERSON"), (44, 60, "GPE")]}),
    ("Microsoft acquired LinkedIn for $26.2 billion.", {"entities": [(0, 9, "ORG"), (17, 24, "ORG")]}),
    ("The meeting is scheduled for July 15, 2023.", {"entities": [(29, 39, "DATE")]}),
    ("Tesla is headquartered in Palo Alto, California.", {"entities": [(0, 5, "ORG"), (27, 37, "GPE"), (39, 49, "GPE")]}),
    ("Eiffel Tower is located in Paris.", {"entities": [(0, 12, "LOC"), (26, 31, "GPE")]}),
    ("Elon Musk founded SpaceX.", {"entities": [(0, 9, "PERSON"), (20, 26, "ORG")]}),
    ("World War II ended in 1945.", {"entities": [(0, 11, "EVENT"), (20, 24, "DATE")]}),
    ("The Mona Lisa was painted by Leonardo da Vinci.", {"entities": [(4, 14, "WORK_OF_ART"), (28, 46, "PERSON")]}),
]

# Load the pre-trained transformer-based model
nlp = spacy.load("en_core_web_trf")

# Create the NER component (this is optional since it's already included in the model)
ner = nlp.get_pipe("ner")

# Add labels to the NER component
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Prepare the training data in .spacy format
doc_bin = DocBin()  # Initialize a DocBin object for saving the training data
for text, annotations in train_data:
    doc = nlp.make_doc(text)  # Create a Doc object
    example = Example.from_dict(doc, annotations)  # Create an Example object
    doc_bin.add(example.reference)  # Add the Doc to the DocBin

# Ensure the output directory exists
output_dir = "./corpus/"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist

# Save the training data in .spacy format
with open(output_dir + "train.spacy", "wb") as f:
    f.write(doc_bin.to_bytes())

# Now, let's train the model with the new data
# Start training
optimizer = nlp.resume_training()
for iteration in range(100):  # Adjust the number of iterations as needed
    random.shuffle(train_data)
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.5)

# Save the updated model
nlp.to_disk("fine_tuned_model")


/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

The config file does not help in ,while the model training is happening . but is have the documentation of the process of the model training (" have the PARAMETER for the model traning" like --> spacy.format[path], training data[path], validation/testing(dev) data)

In [ ]:
import spacy
from spacy.util import Config

# Create a configuration object
base_config = Config().from_str("""
[paths]
train = "./corpus/train.spacy"
dev = "./corpus/dev.spacy"

[components]
ner = {"factory": "ner"}

[training]
batch_size = 10
dropout = 0.5
n_iter = 1000
""")

# Save the configuration to a file
config_path = "base.cfg"
with open(config_path, "w") as config_file:
    config_file.write(base_config.to_str())


In [ ]:
import spacy
from spacy.training import Example
from spacy.util import load_config
import random

# Load the configuration
config = load_config("base.cfg")

# Load the blank model
nlp = spacy.blank("en")

# Create the NER component
ner = nlp.add_pipe("ner", last=True)

# Add labels to the NER component
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Initialize the training loop
optimizer = nlp.begin_training()

# Training loop
for iteration in range(config["training"]["n_iter"]):
    random.shuffle(train_data)
    for text, annotations in train_data:
        doc = nlp.make_doc(text)  # Create Doc
        example = Example.from_dict(doc, annotations)  # Create Example
        nlp.update([example], drop=config["training"]["dropout"])  # Update the model

# Save the trained model
nlp.to_disk("trained_model")


In [ ]:
# Load the trained model
trained_nlp = spacy.load("trained_model")

# Example text for NER
test_text = "Apple is planning to open a new office in London."

# Process the text
doc = trained_nlp(test_text)

# Extract and print named entities
print("Named Entities, Phrases, and Concepts:")
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")


Named Entities, Phrases, and Concepts:
Entity: Apple, Label: ORG


# checking the NER with the complete different paragraph by using "our previous trained model"


In [ ]:
!pip install colorama



In [ ]:
import spacy
from colorama import Fore, Style

# Load the trained model
trained_nlp = spacy.load("trained_model")

# Your new paragraph
text = """

Elon Musk (born June 28, 1971, Pretoria, South Africa) South African-born American entrepreneur who cofounded the electronic-payment firm PayPal and formed SpaceX, maker of launch vehicles and spacecraft. He was also one of the first significant investors in, as well as chief executive officer of, the electric car manufacturer Tesla. In addition, Musk acquired Twitter (later X) in 2022.

Early life
Musk was born to a South African father and a Canadian mother. He displayed an early talent for computers and entrepreneurship. At age 12 he created a video game and sold it to a computer magazine. In 1988, after obtaining a Canadian passport, Musk left South Africa because he was unwilling to support apartheid through compulsory military service and because he sought the greater economic opportunities available in the United States.

PayPal and SpaceX
Musk attended Queen’s University in Kingston, Ontario, and in 1992 he transferred to the University of Pennsylvania, Philadelphia, where he received bachelor’s degrees in physics and economics in 1997. He enrolled in graduate school in physics at Stanford University in California, but he left after only two days because he felt that the Internet had much more potential to change society than work in physics. In 1995 he founded Zip2, a company that provided maps and business directories to online newspapers. In 1999 Zip2 was bought by the computer manufacturer Compaq for $307 million, and Musk then founded an online financial services company, X.com, which later became PayPal, which specialized in transferring money online. The online auction eBay bought PayPal in 2002 for $1.5 billion.

Musk was long convinced that for life to survive, humanity has to become a multiplanet species. However, he was dissatisfied with the great expense of rocket launchers. In 2002 he founded Space Exploration Technologies (SpaceX) to make more affordable rockets. Its first two rockets were the Falcon 1 (first launched in 2006) and the larger Falcon 9 (first launched in 2010), which were designed to cost much less than competing rockets. A third rocket, the Falcon Heavy (first launched in 2018), was designed to carry 117,000 pounds (53,000 kg) to orbit, nearly twice as much as its largest competitor, the Boeing Company’s Delta IV Heavy, for one-third the cost. SpaceX has announced the successor to the Falcon 9 and the Falcon Heavy: the Super Heavy–Starship system. The Super Heavy first stage would be capable of lifting 100,000 kg (220,000 pounds) to low Earth orbit. The payload would be the Starship, a spacecraft designed for providing fast transportation between cities on Earth and building bases on the Moon and Mars. SpaceX also developed the Dragon spacecraft, which carries supplies to the International Space Station (ISS). Dragon can carry as many as seven astronauts, and it had a crewed flight carrying astronauts Doug Hurley and Robert Behnken to the ISS in 2020. The first test flights of the Super Heavy–Starship system launched in 2020. In addition to being CEO of SpaceX, Musk was also chief designer in building the Falcon rockets, Dragon, and Starship. SpaceX is contracted to build the lander for the astronauts returning to the Moon by 2025 as part of NASA’s Artemis space program.

Video released by spacecraft maker SpaceX celebrating its Dragon capsule, which on May 25, 2012, became the first commercial spacecraft to dock with the International Space Station.
SpaceX (A Britannica Publishing Partner)
Tesla
Musk had long been interested in the possibilities of electric cars, and in 2004 he became one of the major funders of Tesla Motors (later renamed Tesla), an electric car company founded by entrepreneurs Martin Eberhard and Marc Tarpenning. In 2006 Tesla introduced its first car, the Roadster, which could travel 245 miles (394 km) on a single charge. Unlike most previous electric vehicles, which Musk thought were stodgy and uninteresting, it was a sports car that could go from 0 to 60 miles (97 km) per hour in less than four seconds. In 2010 the company’s initial public offering raised about $226 million. Two years later Tesla introduced the Model S sedan, which was acclaimed by automotive critics for its performance and design. The company won further praise for its Model X luxury SUV, which went on the market in 2015. The Model 3, a less-expensive vehicle, went into production in 2017 and became the best-selling electric car of all time.

Elon Musk
Elon Musk, 2014.
Jerry Lampen—EPA/Shutterstock.com
Dissatisfied with the projected cost ($68 billion) of a high-speed rail system in California, Musk in 2013 proposed an alternate faster system, the Hyperloop, a pneumatic tube in which a pod carrying 28 passengers would travel the 350 miles (560 km) between Los Angeles and San Francisco in 35 minutes at a top speed of 760 miles (1,220 km) per hour, nearly the speed of sound. Musk claimed that the Hyperloop would cost only $6 billion and that, with the pods departing every two minutes on average, the system could accommodate the six million people who travel that route every year. However, he stated, between running SpaceX and Tesla, he could not devote time to the Hyperloop’s development.

X (formerly Twitter)
Musk joined the social media service Twitter in 2009, and, as @elonmusk, he became one of the most popular accounts on the site, with more than 85 million followers as of 2022. He expressed reservations about Tesla’s being publicly traded, and in August 2018 he made a series of tweets about taking the company private at a value of $420 per share, noting that he had “secured funding.” (The value of $420 was seen as a joking reference to April 20, a day celebrated by devotees of cannabis.) The following month the U.S. Securities and Exchange Commission (SEC) sued Musk for securities fraud, alleging that the tweets were “false and misleading.” Shortly thereafter Tesla’s board rejected the SEC’s proposed settlement, reportedly because Musk had threatened to resign. However, the news sent Tesla stock plummeting, and a harsher deal was ultimately accepted. Its terms included Musk’s stepping down as chairman for three years, though he was allowed to continue as CEO; his tweets were to be preapproved by Tesla lawyers, and fines of $20 million for both Tesla and Musk were levied.

Musk was critical of Twitter’s commitment to principles of free speech, in light of the company’s content-moderation policies. Early in April 2022, Twitter’s filings with the SEC disclosed that Musk had bought more than 9 percent of the company. Shortly thereafter Twitter announced that Musk would join the company’s board, but Musk decided against that and made a bid for the entire company, at a value of $54.20 a share, for $44 billion. Twitter’s board accepted the deal, which would make him sole owner of the company. Musk stated that his plans for the company included “enhancing the product with new features, making the algorithms open source to increase trust, defeating the spam bots, and authenticating all humans.” In July 2022 Musk announced that he was withdrawing his bid, stating that Twitter had not provided sufficient information about bot accounts and claiming that the company was in “material breach of multiple provisions” of the purchase agreement. Bret Taylor, the chair of Twitter’s board of directors, responded by saying that the company was “committed to closing the transaction on the price and terms agreed upon with Mr. Musk.” Twitter sued Musk to force him to buy the company. In September 2022 Twitter’s shareholders voted to accept Musk’s offer. Facing a legal battle, Musk ultimately proceeded with the deal, and it was completed in October.

Among Musk’s first acts as Twitter’s owner were to lay off about half the company and to allow users to purchase for $8 a month the blue check-mark verification, which had previously been bestowed by Twitter upon notable figures. In addition, he disbanded Twitter’s content-moderation body and reinstated many banned accounts, most notably that of former U.S. president Donald Trump, which had been suspended after the U.S. Capitol attack on January 6, 2021. Advertising revenue fell sharply as many companies withdrew their ads from the platform. Musk changed the name of the company from Twitter to X in July 2023. (Tweets became posts with the change.)
"""

# Process the new text
doc = trained_nlp(text)

# Extract and print named entities
print("Named Entities, Phrases, and Concepts:")
for ent in doc.ents:
    print(f"{Fore.BLUE}Entity: {ent.text}, Label: {ent.label_}{Style.RESET_ALL}")

# Extract and print Part-of-Speech tags
print("\nPart-of-Speech Tags:")
for token in doc:
    print(f"Word: {token.text}, POS: {token.pos_}")

# Highlighting entity and label in green
print("\nHighlighted Entities and Labels:")
for ent in doc.ents:
    entity_text = f"{Fore.GREEN}{ent.text}{Style.RESET_ALL}"
    label_text = f"{Fore.GREEN}{ent.label_}{Style.RESET_ALL}"
    print(f"{Fore.BLUE}Entity: {entity_text}, Label: {label_text}{Style.RESET_ALL}")

# Print POS tags without color formatting
#print("\nPart-of-Speech Tags:")
#for token in doc:
    #print(f"Word: {token.text}, POS: {token.pos_}")




Named Entities, Phrases, and Concepts:
Entity: Elon Musk, Label: PERSON
Entity: June 28, Label: PERSON
Entity: , Pretoria, Label: LOC
Entity: , South, Label: LOC
Entity: Africa, Label: ORG
Entity: ) South, Label: LOC
Entity: African-, Label: LOC
Entity: Tesla, Label: ORG
Entity: , Musk, Label: PERSON
Entity: Twitter, Label: ORG
Entity: South African, Label: LOC
Entity: Canadian, Label: ORG
Entity: Canadian, Label: ORG
Entity: , Musk, Label: PERSON
Entity: South Africa, Label: LOC
Entity: United States, Label: LOC
Entity: Queen, Label: ORG
Entity: ’s University, Label: LOC
Entity: Kingston, Label: ORG
Entity: , Ontario, Label: LOC
Entity: ,, Label: ORG
Entity: Pennsylvania,, Label: LOC
Entity: Philadelphia,, Label: LOC
Entity: Stanford University, Label: LOC
Entity: California, Label: GPE
Entity: ,, Label: ORG
Entity: X.com,, Label: PERSON
Entity: PayPal,, Label: PERSON
Entity: However,, Label: LOC
Entity: Space Exploration, Label: LOC
Entity: Technologies, Label: ORG
Entity: Falcon, La